<a href="https://colab.research.google.com/github/RaghavMaskara21/Machine-Learning/blob/main/7_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
!pip install emoji

import re
import nltk
import emoji
from sklearn.metrics import classification_report
import string
nltk.download('words')
words = set(nltk.corpus.words.words())

     |████████████████████████████████| 175 kB 4.9 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=b7a94855f4b972cb8b0ee3f9c94a93645d3baf5b14e88df6d248bc6e7dd1251e
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
data = pd.read_csv("spam.csv", encoding = "latin-1")
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
train = data[0:2000]
test = data[2000:3100] 
print("Shape of training data: ", train.shape,"\nShape of test data: ",  test.shape)
train.head()

Shape of training data:  (2000, 2) 
Shape of test data:  (1100, 2)


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train.v1.append(test.v1))

train['ENCODED_INT'] = le.transform(train.v1)
test['ENCODED_INT'] = le.transform(test.v1)

train.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,v1,v2,ENCODED_INT
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0


In [ ]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet)                                    
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)            
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI)           
    tweet = tweet.replace("#", "").replace("_", " ")                            
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    tweet = re.sub("[^-9A-Za-z ]", "" , tweet)                                  
    tweet = "".join([i.lower() for i in tweet if i not in string.punctuation])  
    return tweet

nltk.download('wordnet')

lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w,pos = 'v') for w in text.split()]

nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(tokens):
    tokens=[word for word in tokens if word not in stopword]
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train['cleaned'] = train['v2'].apply(cleaner)
train['lammatized'] = train['cleaned'].apply(lemmatize_text)
train['cleaned_text'] = train['lammatized'].apply(remove_stopwords)

test['cleaned'] = test['v2'].apply(cleaner)
test['lammatized'] = test['cleaned'].apply(lemmatize_text)
test['cleaned_text'] = test['lammatized'].apply(remove_stopwords)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
nltk.download('averaged_perceptron_tagger')
pos_tags = []

for token in train['cleaned_text']:
  pos_tag = nltk.pos_tag(token)
  pos_tags.append(pos_tag)

nouns_array = []
no_of_noun = []
verbs_array = []
no_of_verbs = []
adj_array = []
no_of_adj = []
for token in pos_tags:
  noun = []
  verb = []
  adj = []
  for i in token:
    if(i[1]=="NN" or i[1]=="NNS" or i[1]=="NNP" or i[1]=="NNPS"):
      noun.append(i[0])
    if(i[1]=="JJR" or i[1]=="JJS"):
      adj.append(i[0])
    if(i[1]=="VB" or i[1]=="VBG" or i[1]=="VBD" or i[1]=="VBN" or i[1]=="VBP" or i[1]=="VBZ"):
      verb.append(i[0])
  nouns_array.append(noun)
  no_of_noun.append(len(noun))
  verbs_array.append(verb)
  no_of_verbs.append(len(verb))
  adj_array.append(adj)
  no_of_adj.append(len(adj))

train['nouns'] = nouns_array
train['no_of_nouns'] = no_of_noun
train['verbs'] = verbs_array
train['no_of_verbs'] = no_of_verbs
train['adjectives'] = adj_array
train['no_of_adj'] = no_of_adj

one_letter_word_count= []
two_letter_word_count= []
def select_one_letter_word(word):
  if(len(word)==1):
    return True
  return False

def select_two_letter_word(word):
  if(len(word)==2):
    return True
  return False

for token in train['cleaned_text']:
  one_letter = list(filter(select_one_letter_word, token))
  two_letter = list(filter(select_two_letter_word, token))
  one_letter_word_count.append(len(one_letter))
  two_letter_word_count.append(len(two_letter))

train["one_letter_word_count"] = one_letter_word_count
train["two_letter_word_count"] = two_letter_word_count

train.head(5)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,v1,v2,ENCODED_INT,cleaned,lammatized,cleaned_text,nouns,no_of_nouns,verbs,no_of_verbs,adjectives,no_of_adj,one_letter_word_count,two_letter_word_count
0,ham,"Go until jurong point, crazy.. Available only ...",0,go until point crazy available only in n gre...,"[go, until, point, crazy, available, only, in,...","[go, point, crazy, available, n, great, world,...","[point, crazy, world, la, buffet, cine, get, wat]",8,"[go, e]",2,[],0,2,2
1,ham,Ok lar... Joking wif u oni...,0,lar u,"[lar, u]","[lar, u]",[u],1,[],0,[],0,1,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry in a to win fa cup final st may ...,"[free, entry, in, a, to, win, fa, cup, final, ...","[free, entry, win, fa, cup, final, st, may, te...","[entry, cup, st, entry, question, rate, c]",7,"[win, text, apply]",3,[],0,1,3
3,ham,U dun say so early hor... U c already then say...,0,u dun say so early u c already then say,"[u, dun, say, so, early, u, c, already, then, ...","[u, dun, say, early, u, c, already, say]","[dun, c]",2,"[say, say]",2,[],0,3,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,i don t think he goes to he around here though,"[i, don, t, think, he, go, to, he, around, her...","[think, go, around, though]",[think],1,[go],1,[],0,0,1


In [ ]:
nltk.download('averaged_perceptron_tagger')
pos_tags = []

for token in test['cleaned_text']:
  pos_tag = nltk.pos_tag(token)
  pos_tags.append(pos_tag)

nouns_array = []
no_of_noun = []
verbs_array = []
no_of_verbs = []
adj_array = []
no_of_adj = []
for token in pos_tags:
  noun = []
  verb = []
  adj = []
  for i in token:
    if(i[1]=="NN" or i[1]=="NNS" or i[1]=="NNP" or i[1]=="NNPS"):
      noun.append(i[0])
    if(i[1]=="JJR" or i[1]=="JJS"):
      adj.append(i[0])
    if(i[1]=="VB" or i[1]=="VBG" or i[1]=="VBD" or i[1]=="VBN" or i[1]=="VBP" or i[1]=="VBZ"):
      verb.append(i[0])
  nouns_array.append(noun)
  no_of_noun.append(len(noun))
  verbs_array.append(verb)
  no_of_verbs.append(len(verb))
  adj_array.append(adj)
  no_of_adj.append(len(adj))

test['nouns'] = nouns_array
test['no_of_nouns'] = no_of_noun
test['verbs'] = verbs_array
test['no_of_verbs'] = no_of_verbs
test['adjectives'] = adj_array
test['no_of_adj'] = no_of_adj

one_letter_word_count= []
two_letter_word_count= []
def select_one_letter_word(word):
  if(len(word)==1):
    return True
  return False

def select_two_letter_word(word):
  if(len(word)==2):
    return True
  return False

for token in test['cleaned_text']:
  one_letter = list(filter(select_one_letter_word, token))
  two_letter = list(filter(select_two_letter_word, token))
  one_letter_word_count.append(len(one_letter))
  two_letter_word_count.append(len(two_letter))

test["one_letter_word_count"] = one_letter_word_count
test["two_letter_word_count"] = two_letter_word_count

test.head(5)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,v1,v2,ENCODED_INT,cleaned,lammatized,cleaned_text,nouns,no_of_nouns,verbs,no_of_verbs,adjectives,no_of_adj,one_letter_word_count,two_letter_word_count
2000,ham,But i'll b going 2 sch on mon. My sis need 2 t...,0,but i b going on mon my sis need take,"[but, i, b, go, on, mon, my, sis, need, take]","[b, go, mon, sis, need, take]","[b, sis]",2,"[go, need, take]",3,[],0,1,1
2001,ham,Idea will soon get converted to live:),0,idea will soon get converted to live,"[idea, will, soon, get, convert, to, live]","[idea, soon, get, convert, live]","[idea, live]",2,[get],1,[],0,0,0
2002,spam,TheMob>Yo yo yo-Here comes a new selection of ...,1,yo yo yo here comes a new selection of hot f...,"[yo, yo, yo, here, come, a, new, selection, of...","[yo, yo, yo, come, new, selection, hot, get, f...","[yo, selection, click, link, send, ur]",6,"[yo, get]",2,[],0,0,4
2003,ham,S....s...india going to draw the series after ...,0,s s going to draw the series after many in s...,"[s, s, go, to, draw, the, series, after, many,...","[go, draw, series, many, south, soil]","[series, soil]",2,[go],1,[],0,0,1
2004,ham,"Goodmorning, today i am late for &lt;DECIMAL&...",0,today i am late for decimal min,"[today, i, be, late, for, decimal, min]","[today, late, decimal, min]","[today, decimal, min]",3,[],0,[],0,0,0


In [ ]:
train.columns

Index(['v1', 'v2', 'ENCODED_INT', 'cleaned', 'lammatized', 'cleaned_text',
       'nouns', 'no_of_nouns', 'verbs', 'no_of_verbs', 'adjectives',
       'no_of_adj', 'one_letter_word_count', 'two_letter_word_count'],
      dtype='object')

In [ ]:
cols = ['one_letter_word_count', 'two_letter_word_count', 'no_of_nouns', 'no_of_verbs', 'no_of_adj']
train_X = train[cols].values
test_X = test[cols].values

train_y = train['ENCODED_INT']
test_y = test['ENCODED_INT']

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=500, tol=1e-6)

clf.fit(np.concatenate((train_X, test_X), axis=0), np.concatenate((train_y, test_y), axis=0))

y_pred = clf.predict(test_X)

from sklearn.metrics import classification_report

print(classification_report(le.inverse_transform(test_y), le.inverse_transform(y_pred)))

              precision    recall  f1-score   support

         ham       0.89      0.98      0.93       961
        spam       0.56      0.20      0.30       139

    accuracy                           0.88      1100
   macro avg       0.73      0.59      0.62      1100
weighted avg       0.85      0.88      0.85      1100



In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

clf.fit(np.concatenate((train_X, test_X), axis=0), np.concatenate((train_y, test_y), axis=0))

y_pred = clf.predict(test_X)

print(classification_report(le.inverse_transform(test_y), le.inverse_transform(y_pred)))

              precision    recall  f1-score   support

         ham       0.91      0.93      0.92       961
        spam       0.43      0.39      0.41       139

    accuracy                           0.86      1100
   macro avg       0.67      0.66      0.66      1100
weighted avg       0.85      0.86      0.85      1100

